In [101]:
import pandas as pd
import hvplot.pandas
import numpy as np
import urllib

In [ ]:
tempe = 'http://data-tempegov.opendata.arcgis.com/datasets/ba76da19a5274301bb8a010483aec14a_0.csv'
scottsdale = 'http://data.scottsdaleaz.gov:8000/spd_PDCallsForServic_2017_1.csv'

In [228]:
def get_scottsdale_cfs(ext='csv'):
    base_url = 'http://data.scottsdaleaz.gov:8000/spd_PDCallsForServic_'
    one = 1
    for year in range(2017, 2020):
        for month in range(1, 13):
            yearmonth = '{year}_{month}.{ext}'.format(year=year, month=month, ext=ext)
            url = base_url + yearmonth
            if one > 0:
                try:
                    df = pd.read_csv(url, parse_dates=['Create Date'])
                    one -= 1
                    month0, year0 = month, year
                except:
                    print(yearmonth, 'doesn\'t exist')
                    continue
            elif one < 1:
                try:
                    df2 = pd.read_csv(url, parse_dates=['Create Date'])
                    df = df.merge(df2, how='outer')
                except:
                    print('Collected', len(df), 'records, from {}/{} up to {}/{}.'.format(month0, year0, month-1, year))
                    df.rename(columns={'Create Date': 'dt'}, inplace=True)
                    df.set_index('dt', inplace=True)
                    break
    return df

def get_chandler_cfs():
    base_url = 'https://data.chandlerpd.com/catalog/calls-for-service-'
    dfs = dict()
    for year in range(2013, 2020):
        year_id = '{year}/download/csv'.format(year=str(year))
        url = base_url + year_id
        if year == 2013:
            df = pd.read_csv(url)
        else:
            df2 = pd.read_csv(url)
            df = df.merge(df2, how='outer')
        print('Completed', year)
    return df

In [ ]:
df = df.merge(df2, how='outer')

In [220]:
dfs = get_chandler_cfs()

Completed 2013
Completed 2014
Completed 2015
Completed 2016
Completed 2017


/Users/tjs/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Completed 2018
Completed 2019


In [226]:
df = dfs.get('2013')
for year in range(2014, 2020):
    df2 = dfs.get(str(year))
    df = df.merge(df2, how='outer')

ValueError: You are trying to merge on object and float64 columns. If you wish to proceed you should use pd.concat

In [225]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1046878 entries, 0 to 1046877
Columns: 162 entries, ID_x to Call Travel Time
dtypes: datetime64[ns](1), float64(75), object(86)
memory usage: 1.3+ GB


In [ ]:
chandler.info()

In [214]:
import urllib
url = 'https://data.chandlerpd.com/catalog/calls-for-service-2019.csv'
data = pd.read_csv(url)

'/var/folders/5y/54k2crn973jd68wj5j3vzf_m0000gn/T/tmp7o65nu2l'

In [186]:
a = range(2013, 2020)
list(a)

[2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [177]:
cdf = pd.read_csv('https://data.chandlerpd.com/catalog/calls-for-service-2013.csv')

HTTPError: HTTP Error 404: Not Found

In [ ]:
def get_tempe_cfs():
    url = 'http://data-tempegov.opendata.arcgis.com/datasets/ba76da19a5274301bb8a010483aec14a_0.csv'
    tempe = pd.read_csv(url, parse_dates=['occ_dt'])
    
    return tempe

def get_mesa_cfs():
    url = 'https://data.mesaaz.gov/api/views/4k95-x7aw/rows.csv'
    mesa = pd.read_csv(url, parse_dates=['Creation Datetime'])
    
    return mesa

def get_gilbert_cfs():
    url = 'https://opendata.arcgis.com/datasets/2dcb4c20c9a444f2a5d3dca180b651c9_2.csv'
    gilbert = pd.read_csv(url, parse_dates=['EventDate'])
    
    return gilbert

In [123]:
df = get_scottsdale_cfs()

Collected 444695 records, from 1/2017 up to 1/2019.


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 444695 entries, 2017-01-15 12:33:02 to 2019-01-20 01:44:46
Data columns (total 13 columns):
Common Event ID              444695 non-null int64
CFS Number                   444695 non-null object
Street Directional Prefix    335623 non-null object
Street Name                  336881 non-null object
Street Type                  320650 non-null object
Cross Street #1              194580 non-null object
Cross Street #2              201810 non-null object
Call Type                    444695 non-null object
Call Sub-type                444695 non-null object
Call Type English            440100 non-null object
Call Sub-type English        139877 non-null object
CFS_Type                     444695 non-null object
CFS_English                  444695 non-null object
dtypes: int64(1), object(12)
memory usage: 47.5+ MB


In [168]:
def get_tempe_cfs():
    url = 'http://data-tempegov.opendata.arcgis.com/datasets/ba76da19a5274301bb8a010483aec14a_0.csv'
    tempe = pd.read_csv(url, parse_dates=['occ_dt'])
    
    return tempe

In [169]:
def get_mesa_cfs():
    url = 'https://data.mesaaz.gov/api/views/4k95-x7aw/rows.csv'
    mesa = pd.read_csv(url, parse_dates=['Creation Datetime'])
    
    return mesa

In [170]:
def get_gilbert_cfs():
    url = 'https://opendata.arcgis.com/datasets/2dcb4c20c9a444f2a5d3dca180b651c9_2.csv'
    gilbert = pd.read_csv(url, parse_dates=['EventDate'])
    
    return gilbert

In [ ]:
tempe = get_tempe_cfs()
mesa = get_mesa_cfs()
gilbert = get_gilbert_cfs()

In [174]:
gdf = pd.read_json('https://opendata.arcgis.com/datasets/2dcb4c20c9a444f2a5d3dca180b651c9_2.geojson')

In [175]:
gdf.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'properties': {'PoliceEven..."
1,FeatureCollection,"{'type': 'Feature', 'properties': {'PoliceEven..."
2,FeatureCollection,"{'type': 'Feature', 'properties': {'PoliceEven..."
3,FeatureCollection,"{'type': 'Feature', 'properties': {'PoliceEven..."
4,FeatureCollection,"{'type': 'Feature', 'properties': {'PoliceEven..."


In [ ]:
df.rename(columns={'occ_dt': 'dt'}, inplace=True)
df.set_index('dt', inplace=True)

In [ ]:
def get_phoenix_cfs():
    'https://www.phoenixopendata.com/dataset/64a60154-3b2d-4583-8fb5-6d5e1b469c28/resource/1d536ee6-7ffb-49c3-bffe-5cdd98a3c97e/download/calls-for-service_calls-for-service_callsforservice.csv'

CPU times: user 1.49 s, sys: 273 ms, total: 1.77 s
Wall time: 7.49 s


In [163]:
mesa.head()

,Event Number,Creation Datetime,Creation Month,Creation Year,Event Type Code,Event Type Description,Call Priority,Event Address,Street Number,Street Direction,Street Name,Street Type,Latitude,Longitude,Dispatch Group,Division,Response Time Seconds,Response Time Minutes,Location 1
0,P20170010004,01/01/2017 12:02:00 AM,Jan,2017,900,WELFARE CHECK,P3,400 Block of N LESUEUR MESA,400.0,N,LESUEUR,NaN,33.422611,-111.820437,P2,Central,69,1.15,"(33.422611, -111.820437)"
1,P20170010005,01/01/2017 12:03:00 AM,Jan,2017,921,PROWLER,P1,1200 Block of W 6TH DR MESA,1200.0,W,6TH,DR,33.402687,-111.857332,P1,Fiesta,427,7.12,"(33.402687, -111.857332)"
2,P20170010008,01/01/2017 12:05:00 AM,Jan,2017,666,SUICIDE,P1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P4,Superstition,558,9.30,NaN
3,P20170010010,01/01/2017 12:06:00 AM,Jan,2017,961,ACCIDENT,P1,1600 Block of S DOBSON RD MESA,1600.0,S,DOBSON,RD,33.384051,-111.875400,P1,Fiesta,43,0.72,"(33.384051, -111.8754)"
4,P20170010015,01/01/2017 12:09:00 AM,Jan,2017,421,SHOTS FIRED,P1,1700 Block of S SOSSAMAN RD MESA,1700.0,S,SOSSAMAN,RD,33.383873,-111.667177,P4,Superstition,151,2.52,"(33.383873, -111.667177)"


In [156]:
len(mesa)

TypeError: object of type 'JsonReader' has no len()

In [61]:
df = pd.read_csv('callsforservice.csv')
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,CALL_RECEIVED,HUNDREDBLOCKADDR,GRID
0,201502108490,N,NO ACTION REQUIRED,647V,SUSPICIOUS PERSON IN VEHICLE,11/2/2015 9:16:27AM,154XX N 29TH ST,DF34
1,201502108749,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 9:58:57AM,96XX N 10TH AVE,CH26
2,201502108648,O,OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY),459A,BURGLARY ALARM,11/2/2015 9:42:42AM,38XX E WATKINS ST,AH36B
3,201502108757,N,NO ACTION REQUIRED,418T,TRESPASSING,11/2/2015 10:02:11AM,36XX W CAMELBACK RD,BI20
4,201502108592,N,NO ACTION REQUIRED,900,CHECK WELFARE,11/2/2015 9:30:53AM,40XX N 44TH AVE,BG18


In [62]:
with open('apikey.txt', 'r') as door:
    key = door.read()

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2192756 entries, 0 to 2192755
Data columns (total 8 columns):
INCIDENT_NUM        int64
DISP_CODE           object
DISPOSITION         object
FINAL_RADIO_CODE    object
FINAL_CALL_TYPE     object
CALL_RECEIVED       object
HUNDREDBLOCKADDR    object
GRID                object
dtypes: int64(1), object(7)
memory usage: 133.8+ MB


In [64]:
# tst = df.iloc[:500, :]
# tst.head()

In [65]:
df['HUNDREDBLOCKADDR'] = df.HUNDREDBLOCKADDR.str.replace('XX', '00')
df.dropna(inplace=True)
df = df[df.FINAL_RADIO_CODE.isin(['961', '962', '963', '961H', '962H', '963H'])]
df.loc[:, 'CALL_RECEIVED'] = pd.to_datetime(df.CALL_RECEIVED)
df.set_index('CALL_RECEIVED', inplace=True)

In [66]:
df['month'] = df.index.month
df['day'] = df.index.dayofweek
df['hour'] = df.index.hour

In [67]:
df.head()

,INCIDENT_NUM,DISP_CODE,DISPOSITION,FINAL_RADIO_CODE,FINAL_CALL_TYPE,HUNDREDBLOCKADDR,GRID,month,day,hour
CALL_RECEIVED,,,,,,,,,,
2015-11-01 01:01:07,201502100315,N,NO ACTION REQUIRED,961H,HIT & RUN ACCIDENT NO INJURY,S 67TH AVE & W LOWER BUCKEYE RD,AG13,11,6,1
2015-11-02 08:04:44,201502108062,A,ACCIDENT REPORT,962,ACCIDENT WITH INJURIES,N 49TH DR & W ESCUDA DR,EA17,11,0,8
2015-11-01 10:09:50,201502102088,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,E PARADISE LN & N 32ND ST,DF35,11,6,10
2015-11-01 10:35:06,201502102198,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,4000 W CAMELBACK RD,BI19A,11,6,10
2015-11-01 18:03:38,201502104894,N,NO ACTION REQUIRED,961,ACCIDENT NO INJURIES,3700 W SIERRA ST,DA20,11,6,18


In [68]:
df['ishitrun'] = df.FINAL_RADIO_CODE.isin(['961H', '962H', '963H']).astype(np.int)

In [69]:
hracs = df[df.ishitrun == 1]
lacs = df[df.ishitrun == 0]

In [70]:
lacs.month.hvplot.hist(color='navy').options(alpha=0.7) * hracs.month.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [month]   (month_count)
   .Histogram.II :Histogram   [month]   (month_count)

In [71]:
lacs.day.hvplot.hist(color='navy').options(alpha=0.7) * hracs.day.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [day]   (day_count)
   .Histogram.II :Histogram   [day]   (day_count)

In [72]:
lacs.hour.hvplot.hist(color='navy').options(alpha=0.7) * hracs.hour.hvplot.hist(color='red').options(alpha=0.7)

:Overlay
   .Histogram.I  :Histogram   [hour]   (hour_count)
   .Histogram.II :Histogram   [hour]   (hour_count)

In [73]:
lacs.DISPOSITION.value_counts() / len(lacs) * 100

ACCIDENT REPORT                           61.493233
NO ACTION REQUIRED                        31.057541
DEPARTMENTAL REPORT                        6.999781
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.228210
TRAFFIC CITATION                           0.221235
Name: DISPOSITION, dtype: float64

In [74]:
hracs.DISPOSITION.value_counts() / len(hracs) * 100

NO ACTION REQUIRED                        51.400935
DEPARTMENTAL REPORT                       24.767106
ACCIDENT REPORT                           23.278724
OTHER (SUPP/FALSE ALARM/FI/LOUD PARTY)     0.478281
TRAFFIC CITATION                           0.074954
Name: DISPOSITION, dtype: float64

In [75]:
lacs.FINAL_CALL_TYPE.value_counts() / len(lacs) * 100

ACCIDENT NO INJURIES      81.209017
ACCIDENT WITH INJURIES    18.576725
ACCIDENT FATALITY          0.214259
Name: FINAL_CALL_TYPE, dtype: float64

In [76]:
hracs.FINAL_CALL_TYPE.value_counts() / len(hracs) * 100

HIT & RUN ACCIDENT NO INJURY        93.550344
HIT & RUN ACCIDENT WITH INJURIES     6.356855
HIT & RUN  FATALITY                  0.092801
Name: FINAL_CALL_TYPE, dtype: float64

In [77]:
len(hracs) / (len(hracs) + len(lacs))

0.2182638299198367

In [78]:
df.GRID.value_counts()

BG27     705
BA27     624
BG13     603
BC19A    590
BI23B    569
DG29     553
CI22     529
BC17A    528
CA21     522
BE11     519
BG19     516
BG17     516
BE31     510
AB16     503
BI31B    485
BE38     484
BI29     484
BD11     483
BG29     479
BG11     474
BE17     460
BI32     458
BC13A    458
BE35     449
CA25     434
BC29A    426
BG21A    418
CA23B    416
BI21     414
CG21     404
        ... 
HA16       1
EH14       1
IH29       1
EF39       1
EE28       1
HA17       1
JA28       1
DE30       1
EI18       1
JA27       1
AJ10       1
FI37       1
CI20B      1
AF08       1
EE40       1
JC22       1
JD26       1
EH34       1
AE14       1
EI15       1
FG42       1
AC10       1
CC18       1
GC18       1
FC13       1
EF34       1
GF12       1
EI34       1
IC12       1
EI17       1
Name: GRID, Length: 1669, dtype: int64